In [1]:
%reload_ext autoreload
%autoreload 2

In [8]:
from app.helper import q2f
from app import create_app
from datetime import datetime, timedelta
import pytz
from tinkoff.invest import Client, CandleInterval
from app.config import AppConfig

app = create_app()
with app.app_context():
    
    def get_current_price_from_last_candle(figi: str) -> float:
        """
        Получает текущую цену для заданного FIGI, используя данные последней свечи с интервалом 1 минута.
        
        :param figi: Идентификатор инструмента (FIGI).
        :return: Текущая цена (float) на основе цены закрытия последней свечи.
        """
        token = AppConfig.TOKEN  # Ваш токен
        # Используем pytz для работы с часовыми поясами, например UTC
        now = datetime.now(pytz.UTC)
        # Запрашиваем свечи за последние 5 минуты, чтобы гарантировать наличие свежих данных
        start_time = now - timedelta(minutes=5)
        
        with Client(token) as client:
            candles_response = client.market_data.get_candles(
                figi=figi,
                from_=start_time,
                to=now,
                interval=CandleInterval.CANDLE_INTERVAL_1_MIN
            )
            # Проверяем, что свечи получены
            if candles_response.candles:
                last_candle = candles_response.candles[-1]
                # Предполагается, что функция q2f корректно преобразует Quotation в float
                return q2f(last_candle.close)
            else:
                raise Exception("No candle data available for FIGI: " + figi)
    
    # Пример использования:
    figi = "BBG004S68CP5"
    current_price = get_current_price_from_last_candle(figi)
    print(f"Текущая цена: {current_price:.2f}")


Текущая цена: 114.75
